In [1]:
!pip install --upgrade pyarrow
!pip install --upgrade pandas
from google.colab import drive
from os.path import join
from bs4 import BeautifulSoup as bs
import pandas as pd
import pyarrow.feather as feather
import os
import pickle
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.stem.porter import *
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
import string
import re
from nltk.stem.porter import *
import sqlite3

In [ ]:
ROOT = '/content/drive/'            # This is how you get to the "root" folder of your google drive. 
BASE = 'My Drive/Umar - Omdena Newsroom /' # This is where you specify the subfolder that is the working folder for this notebook. 
PROJECT_PATH = join(ROOT,BASE)

drive.mount(ROOT)
%cd '{PROJECT_PATH}'

In [ ]:
conn = sqlite3.connect(PROJECT_PATH + 'data/omdena_master.db')
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

# Unlabelled Dataset

## Load the feather files into one dataframe.

First we will load the unlabelled dataset provided by Omdena. These are the 13 or so feather files. Each will be loaded up and added to a dataframe. The end result (df_unlab) is a dataset with all the articles in one frame.

In [11]:
#Load the unlabbeled data from the Feather files. 

unlab_data_path = PROJECT_PATH + 'data/raw_unlabeled/'

df_unlab=pd.DataFrame()
for file in os.listdir(unlab_data_path):
    df_tmp = pd.read_feather(unlab_data_path + file)
    df_unlab = df_unlab.append(df_tmp)

df_unlab.reset_index(inplace=True, drop=True)
df_unlab.to_csv(PROJECT_PATH + 'data/raw_combined_articles.csv')
#df_unlab.to_sql('raw_dataset',conn)

## Extract text from HTML

In [26]:
# Clean URLS to extract source

def strip_html (html_string):
    text = html_string[1:-1]
    text = bs(text).get_text()
    return text

def strip_url(string_in,suffix):
    url_string = string_in
    for suf in suffix:
        if suf in url_string:
            pos = url_string.find(suf)
            url_string = url_string[:pos]
    
    url_string = url_string.replace('https://','')
    url_string = url_string.replace('http://','')
    url_string = url_string.replace('www.','')
    return url_string

df_unlab['title'] = df_unlab['article_title'].apply(strip_html,1)
df_unlab['subtitle'] = df_unlab['article_subtitle'].apply(strip_html,1)
df_unlab['text'] = df_unlab['article_text'].apply(strip_html,1)
df_unlab['author'] = df_unlab['author_name'].apply(strip_html,1)

suffs = ['.com','.org','.co.uk','.net']
df_unlab['source'] = df_unlab['link'].apply(strip_url,1,args=(suffs,))

df_unlab.drop(['article_title','article_subtitle','article_text','author_name'], axis =1, inplace=True)

df_unlab.reset_index()
df_unlab.to_csv(PROJECT_PATH + 'data/cleaned_unlab.csv')
df_unlab.to_sql('cleaned',conn,if_exists='replace')
conn.commit()

df_unlab.head()


nytimes                    7148
cnn                        7148
breitbart                  7148
bbc                        7148
nypost                     7147
timesofindia.indiatimes    7147
foxnews                    7147
chicagotribune             7147
reuters                    7147
aljazeera                  7147
cbsnews                    7147
businessinsider            7147
nbcnews                    7147
latimes                    7147
nationalreview             7147
newsweek                   7147
theguardian                7146
thesun                     7146
thehill                    7146
telegraph                  7146
independent                7146
vox                        7146
vice                       7146
politico                   7146
washingtontimes            7146
dailymail                  7146
boingboing                 7143
usatoday                   7141
apnews                     7096
dailycaller                6604
wired                      6146
zerohedg